# SIT796 Task 3.1D
Brenton Adey
222165064

In [19]:
import numpy as np
from numpy import pi, sin, cos

import random
from math import floor

# Define Dynamics of Acrobot

In [5]:
# Define constants of for Acrobot Components
dt = 0.2
LINK_LENGTH_1 = 1.0  # [m]
LINK_LENGTH_2 = 1.0  # [m]
LINK_MASS_1 = 1.0  #: [kg] mass of link 1
LINK_MASS_2 = 1.0  #: [kg] mass of link 2
LINK_COM_POS_1 = 0.5  #: [m] position of the center of mass of link 1
LINK_COM_POS_2 = 0.5  #: [m] position of the center of mass of link 2
LINK_MOI = 1.0  #: moments of inertia for both links
MAX_VEL_1 = 4 * pi
MAX_VEL_2 = 9 * pi

In [11]:
def dsdt(state_augemented):
    m1 = LINK_MASS_1
    m2 = LINK_MASS_2
    l1 = LINK_LENGTH_1
    lc1 = LINK_COM_POS_1
    lc2 = LINK_COM_POS_2
    I1 = LINK_MOI
    I2 = LINK_MOI
    g = 9.8
    a = state_augemented[-1]
    s = state_augemented[:-1]
    theta1 = s[0]
    theta2 = s[1]
    dtheta1 = s[2]
    dtheta2 = s[3]
    d1 = (
        m1 * lc1**2
        + m2 * (l1**2 + lc2**2 + 2 * l1 * lc2 * cos(theta2))
        + I1
        + I2
    )
    d2 = m2 * (lc2**2 + l1 * lc2 * cos(theta2)) + I2
    phi2 = m2 * lc2 * g * cos(theta1 + theta2 - pi / 2.0)
    phi1 = (
        -m2 * l1 * lc2 * dtheta2**2 * sin(theta2)
        - 2 * m2 * l1 * lc2 * dtheta2 * dtheta1 * sin(theta2)
        + (m1 * lc1 + m2 * l1) * g * cos(theta1 - pi / 2)
        + phi2
    )
    
    ddtheta2 = (a + d2 / d1 * phi1 - phi2) / (m2 * lc2**2 + I2 - d2**2 / d1)
    ddtheta1 = -(d2 * ddtheta2 + phi1) / d1
    
    return dtheta1, dtheta2, ddtheta1, ddtheta2, 0.0

def rk4(derivs, y0, t):
    """
    Integrate 1-D or N-D system of ODEs using 4-th order Runge-Kutta.
    Example for 2D system:
        >>> def derivs(x):
        ...     d1 =  x[0] + 2*x[1]
        ...     d2 =  -3*x[0] + 4*x[1]
        ...     return d1, d2
        >>> dt = 0.0005
        >>> t = np.arange(0.0, 2.0, dt)
        >>> y0 = (1,2)
        >>> yout = rk4(derivs, y0, t)
    Args:
        derivs: the derivative of the system and has the signature ``dy = derivs(yi)``
        y0: initial state vector
        t: sample times
    Returns:
        yout: Runge-Kutta approximation of the ODE
    """

    try:
        Ny = len(y0)
    except TypeError:
        yout = np.zeros((len(t),), np.float_)
    else:
        yout = np.zeros((len(t), Ny), np.float_)

    yout[0] = y0

    for i in np.arange(len(t) - 1):

        this = t[i]
        dt = t[i + 1] - this
        dt2 = dt / 2.0
        y0 = yout[i]

        k1 = np.asarray(derivs(y0))
        k2 = np.asarray(derivs(y0 + dt2 * k1))
        k3 = np.asarray(derivs(y0 + dt2 * k2))
        k4 = np.asarray(derivs(y0 + dt * k3))
        yout[i + 1] = y0 + dt / 6.0 * (k1 + 2 * k2 + 2 * k3 + k4)
    # We only care about the final timestep and we cleave off action value which will be zero
    return yout[-1][:4]

def wrap(x, m, M):
    """Wraps ``x`` so m <= x <= M; but unlike ``bound()`` which
    truncates, ``wrap()`` wraps x around the coordinate system defined by m,M.\n
    For example, m = -180, M = 180 (degrees), x = 360 --> returns 0.
    Args:
        x: a scalar
        m: minimum possible value in range
        M: maximum possible value in range
    Returns:
        x: a scalar, wrapped
    """
    diff = M - m
    while x > M:
        x = x - diff
    while x < m:
        x = x + diff
    return x

def bound(x, m, M=None):
    """Either have m as scalar, so bound(x,m,M) which returns m <= x <= M *OR*
    have m as length 2 vector, bound(x,m, <IGNORED>) returns m[0] <= x <= m[1].
    Args:
        x: scalar
        m: The lower bound
        M: The upper bound
    Returns:
        x: scalar, bound between min (m) and Max (M)
    """
    if M is None:
        M = m[1]
        m = m[0]
    # bound x between min (m) and Max (M)
    return min(max(x, m), M)

In [12]:
def apply_torque(state, torque):
    state_augemented = np.append(state,torque)
    new_state = rk4(dsdt, state_augemented, [0,dt])

    new_state[0] = wrap(new_state[0], -pi, pi)
    new_state[1] = wrap(new_state[1], -pi, pi)
    new_state[2] = bound(new_state[2], -MAX_VEL_1, MAX_VEL_1)
    new_state[3] = bound(new_state[3], -MAX_VEL_2, MAX_VEL_2)
    
    return new_state

In [14]:
theta1 = random.uniform(-0.1,0.1)
theta2 = random.uniform(-0.1,0.1)
dtheta1 = random.uniform(-0.1,0.1)
dtheta2 = random.uniform(-0.1,0.1)

init_state = np.array([theta1,theta2,dtheta1,dtheta2])

new_state = apply_torque(init_state, 1)

# Discretise Acrobot State Space

In [75]:
# Δθ = 20˚ = π/9rad
# Δdθ = 0.5rad/s
# Number of states = 18*18*51*114 = 1,883,736

def discretise(state, D_angle=(pi/9), D_velocity=0.5):
    state[0] = round(state[0]/(D_angle))*(D_angle)
    state[1] = round(state[1]/(D_angle))*(D_angle)

    state[2] = round(state[2]/(D_velocity))*(D_velocity)
    state[3] = round(state[3]/(D_velocity))*(D_velocity)

    return state

In [79]:
# Example discretise function
discretise([4.5,0.5,5.523,-12.008])

[4.537856055185257, 0.3490658503988659, 5.5, -12.0]

In [78]:
# Test Function to ensure dynamics can still operate
state = [0,0,0,0]
for _ in range(0,1000):
    state = discretise(apply_torque(state,random.choice([-1,0,1])))
    print(state)

[0. 0. 0. 0.]
[ 0.   0.   0.  -0.5]
[0. 0. 0. 0.]
[ 0.   0.   0.  -0.5]
[ 0.   0.   0.  -0.5]
[ 0.  0.  0. -1.]
[ 0.         -0.34906585  0.         -1.        ]
[ 0.        -0.6981317  0.        -1.       ]
[ 0.        -0.6981317  0.        -0.5      ]
[ 0.        -0.6981317  0.         0.       ]
[ 0.        -0.6981317  0.         0.5      ]
[ 0.        -0.6981317  0.         1.       ]
[ 0.         -0.34906585  0.          1.5       ]
[0.  0.  0.  1.5]
[0.         0.34906585 0.         1.5       ]
[0.        0.6981317 0.        0.5      ]
[0.        0.6981317 0.        0.       ]
[ 0.         0.6981317  0.        -0.5      ]
[ 0.         0.6981317  0.        -1.       ]
[ 0.          0.34906585  0.         -1.5       ]
[ 0.  0.  0. -2.]
[ 0.         -0.34906585  0.         -2.        ]
[ 0.        -0.6981317 -0.5       -1.       ]
[ 0.        -0.6981317 -0.5       -0.5      ]
[ 0.        -0.6981317 -0.5        0.5      ]
[ 0.        -0.6981317 -0.5        1.       ]
[ 0.         -0.